<a href="https://colab.research.google.com/github/Ivan-KKH/APAI4011_GP/blob/master/predicting_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [177]:
from google.colab import files
import numpy as np
import pandas as pd
import io
import re
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Bidirectional
import warnings
import random
warnings.filterwarnings("ignore")

In [ ]:
uploaded = files.upload()

Saving amazon.csv to amazon (1).csv


In [ ]:
#read csv file
df = pd.read_csv(io.BytesIO(uploaded['amazon.csv']))
#dfReview for review title and review content
dfText = df[['product_name','about_product','category']]

In [ ]:
for column in range(2):
  dfText.iloc[:,column] = dfText.iloc[:,column].apply(lambda x: x.lower())
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub('[^a-zA-z0-9\s]'," ",x)))
dfText['category'] = dfText['category'].apply(lambda x: x.split('|',1)[0])

#combine product_name and about_product
dfText['combined'] = dfText['product_name'] + ' ' + dfText['about_product']

In [ ]:
#tokenize, build vocab, and add paddings on product_name
tokenizer = Tokenizer(num_words=2000,split=' ')
tokenizer.fit_on_texts(dfText['combined'].values)
text = tokenizer.texts_to_sequences(dfText['combined'].values)
print(text)
text = pad_sequences(text)

#category
category_list = []
for categ in dfText['category']:
  if categ not in category_list:
    category_list.append(categ)
print(f'category list: {category_list}')
category = pd.get_dummies(dfText['category'], columns=category_list).values
category_list = ['Car & Motorbike', 'Computers & Accessories', 'Electronics', 'Health & Personal Care', 'Home & Kitchen', 'Home Improvement', 'Musical Instruments', 'Office Products', 'Toys & Games']

print(f'shape of text: {text.shape}')
print(f'shape of category: {category.shape}')


[[1017, 204, 153, 11, 3, 321, 33, 22, 1, 51, 91, 14, 34, 5, 88, 219, 90, 144, 60, 55, 75, 56, 28, 78, 163, 63, 113, 20, 749, 385, 6, 10, 309, 29, 186, 34, 4, 88, 90, 144, 60, 837, 88, 55, 55, 92, 88, 75, 75, 92, 88, 728, 728, 92, 88, 56, 56, 92, 88, 28, 1348, 1991, 1270, 78, 63, 78, 163, 10, 15, 78, 113, 10, 15, 20, 677, 677, 101, 1, 65, 293, 31, 33, 45, 51, 91, 16, 21, 45, 1, 91, 729, 58, 7, 591, 44, 34, 4, 64, 22, 1234, 150, 136, 22, 1599, 18, 30, 645, 322, 79, 204, 153, 39, 4, 176, 566, 611, 1, 1906, 204, 970, 1471, 496, 2, 137, 16, 444, 322, 1, 1530, 7, 919, 3, 177, 1235, 29, 856, 408, 16, 12, 167, 3, 587, 426, 8, 83, 26, 1600, 1601, 558, 241, 242, 635, 1907, 1091, 224, 1172, 698, 1271, 1472, 25, 90, 359, 25, 1, 344, 255, 1306, 262, 2, 89, 68, 6, 8, 145, 260, 13, 1307, 64, 18, 1149, 118, 730, 315, 1432, 3, 310, 554], [995, 762, 971, 230, 33, 22, 10, 1092, 153, 24, 19, 14, 5, 280, 330, 427, 77, 24, 19, 31, 452, 103, 365, 51, 91, 158, 45, 20, 36, 35, 34, 4, 40, 24, 19, 373, 31, 42, 1

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Flatten, Reshape, SpatialDropout1D
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
#split data
X_train, X_test, Y_train, Y_test = train_test_split(text, category, test_size=0.2, random_state=42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

#apply Bidirectional LSTM
model = Sequential()
model.add(Embedding(2000, 128, input_length = 424))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(196, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(9,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

#Train the model for 5 epoches with embedding matrix you obtained earlier.
#print training loss and training accuracy for each epoch
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test,Y_test))  


(1172, 424) (1172, 9)
(293, 424) (293, 9)
Epoch 1/10
37/37 [==============================] - 104s 3s/step - loss: 1.4114 - accuracy: 0.3481 - val_loss: 1.2616 - val_accuracy: 0.3447
Epoch 2/10
37/37 [==============================] - 100s 3s/step - loss: 1.2012 - accuracy: 0.3848 - val_loss: 1.1864 - val_accuracy: 0.3618
Epoch 3/10
37/37 [==============================] - 98s 3s/step - loss: 1.0368 - accuracy: 0.6015 - val_loss: 1.2471 - val_accuracy: 0.3754
Epoch 4/10
37/37 [==============================] - 101s 3s/step - loss: 0.6356 - accuracy: 0.7824 - val_loss: 0.6314 - val_accuracy: 0.8123
Epoch 5/10
37/37 [==============================] - 102s 3s/step - loss: 0.3472 - accuracy: 0.8771 - val_loss: 0.3675 - val_accuracy: 0.8840
Epoch 6/10
37/37 [==============================] - 102s 3s/step - loss: 0.2109 - accuracy: 0.9317 - val_loss: 0.4201 - val_accuracy: 0.8840
Epoch 7/10
37/37 [==============================] - 102s 3s/step - loss: 0.1400 - accuracy: 0.9582 - val_loss: 0.

In [ ]:
#a simple predictor function
def predict(product_name, about_product):
    # Convert to lowercase
    name = product_name.lower()
    desc = about_product.lower()

    #remove punctuation
    name = re.sub(r"http\S+"," ",name)
    desc = re.sub(r"http\S+"," ",desc)

    #text
    text = name + ' ' + desc

    #tokenize, build vocab, and add paddings on product_name
    text = tokenizer.texts_to_sequences(text)
    token_list = []
    for element in text:
        if element:
            token_list.append(element[0])
    token_list = token_list[:423]
    token_len = len(token_list)

    #pad sequence
    zero_list = np.zeros(424 - token_len)
    padded_list = [*zero_list, *token_list]
    padded_list = [padded_list]

    #predict category
    pred = model.predict(padded_list)
    max_idx = np.argmax(pred)
    predicted_category = category_list[max_idx]

    return predicted_category

#choose a random product
rand_idx = random.randint(0,1465)
product_name = df['product_name'][rand_idx]
about_product = df['about_product'][rand_idx]

#predict its category
pred_categ = predict(product_name, about_product)
actual_category = np.argmax(category[rand_idx])
print(f"Product name: {product_name}")
print(f"Product description: {about_product}")
print(f'Predicted Category: {pred_categ}')
print(f'Actual Category: {category_list[actual_category]}')



424
1/1 [==============================] - 0s 176ms/step
Product name: Fire-Boltt Ninja Call Pro Plus 1.83" Smart Watch with Bluetooth Calling, AI Voice Assistance, 100 Sports Modes IP67 Rating, 240*280 Pixel High Resolution
Product description: Fire-Boltt is India' No 1 Wearable Watch Brand Q122 by IDC Worldwide quarterly wearable device tracker Q122.【Bluetooth Calling Watch】- Fire-Boltt Ninja Call Pro Plus Smart Watch enables you to make and receive calls directly from your watch via the built-in speaker and microphone. This smartwatch features a dial pad, option to access recent calls & sync your phone’s contacts.|【1.83" HD Display Smartwatch】- The 46.48mm (1.83-inch) HD display makes the display clear and true-to-life, with vivid colours ensuring smooth readability and keeping the watch as exquisite to look at as when you first lay your eyes on it.|【AI Voice Assistant】- With built-in Voice assistant, you can simply speak to the smartwatch & get things done on your command|【100 Spor